In [ ]:
import socket
import socketserver, time
from threading import Thread

In [ ]:
host = "25.92.87.184"  #http://25.94.218.230:555
port = 1234
timeout = 60
buffer_size = 1024

aux = 0
data_list = []

request = 'data request'
broadcast = ('<broadcast>', 1234)
no_data = bytes('no_data', 'utf-8')
messages = []

In [ ]:
def setup():
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
    
    server.settimeout(timeout)
    server.bind((host, port))

    print('Server rodando')
    
    return server

In [ ]:
def send_request(server):
    
    request = bytes('r', 'utf-8')
    
    server.sendto(request, broadcast)
    
    print('Request sent')

In [ ]:
def receive_message():
    
    msg, address = server.recvfrom(buffer_size)
    message = msg.decode(encoding = 'utf-8')
    
    data_list.append((msg, address))

In [ ]:
def receive_msg(server):
    
    confirmation = bytes('Data processed', 'utf-8')
    
    bytes_received, address = server.recvfrom(buffer_size)
    
    message = bytes_received.decode()
    print('message: ', message)
    print('from: ', address)
        
    if message[0] == 'S':

        bytes_to_send = bytes(message[1:], 'utf-8')
        messages.append(bytes_to_send)
        
        server.sendto(confirmation, address)
        
    elif message[0] == 'A':
        
        if len(messages) != 0:
            to_send = messages[0]
            messages.pop(0)
            server.sendto(to_send, address)
        else:
            server.sendto(no_data, address)
            
    else:
        print('Cant interpretate request')

In [ ]:
server = setup()

In [ ]:
while True:
    receive_msg(server)
    
    time.sleep(1)

In [ ]:
messages = []

In [24]:
help(server)

Help on socket in module socket object:

class socket(_socket.socket)
 |  socket(family=-1, type=-1, proto=-1, fileno=None)
 |  
 |  A subclass of _socket.socket adding the makefile() method.
 |  
 |  Method resolution order:
 |      socket
 |      _socket.socket
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, family=-1, type=-1, proto=-1, fileno=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Wrap __repr__() to reveal the real class name and socket
 |      address(es).
 |  
 |  accept(self)
 |      accept() -> (socket object, address info)
 |      
 |      Wait for an incoming connection.  Return a new socket
 |      representing the connection, and the address of the client.
 |      For IP sockets, the address info is a pair (hostaddr, port).
 |  
 |  close(self)
 |      close()
 |      
 |      Close the socke